In [1]:
#Importando as bibliotecas que usarei: Pandas, Docx, (Datetime e Time?)
import pandas as pd
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT

In [2]:
#Criar Classe PF
class PF:

    #Busca no Banco de Dados pelo parametro 'busca' na 'coluna' (parametro opcional, padrão "xNOMEx")
    def __init__(self, busca, coluna='xNOMEx', aba="PF"):
        
        #Carrega o BD (fornecer nome do arquivo excel e nome da aba da planilha) e converter o BD em string
        self.bd = pd.read_excel("0. Cadastro.xlsm", aba).astype(str)
              
        #retorna a linha com a busca e o index, indice 0 para retornar apenas o numero da linha
        self.linha = self.bd[self.bd[coluna] == busca].index[0]

        #cria o dicionário, sendo as colunas as chaves (a linha é o indice 0 da lista que retornou a "busca")
        self.dicionario = dict(self.bd.iloc[self.linha])
        
        #adiciona o atributo endereço ao dicionário
        self._endereco()
        #adiciona o atributo cliente ao dicionário
        self._qualificacao()
        
    #FUNÇÕES PARA SEREM USADAS INTERNAMENTE NA CLASSE
   
    def _endereco(self):
        #junta tudo em um só atributo
        if self.dicionario['xCOMPLEMENTOx'] != 'nan':
            self.dicionario['xNUMEROx'] = self.dicionario['xNUMEROx'] + " - " + self.dicionario['xCOMPLEMENTOx']

        self.dicionario["xENDERECOx"] = self.dicionario['xLOGRADOUROx'] + ", nº " + self.dicionario['xNUMEROx'] + ", Bairro "\
            + self.dicionario['xBAIRROx'] + ", CEP " + self.dicionario['xCEPx'] + ", " + self.dicionario['xCIDADEx'] + " - " + self.dicionario['xUFx']
      
    def _qualificacao(self):
        #deixa a qualificação pronta e adiciona ao dicionário
        self.dicionario["xCLIENTEx"] = self.dicionario['xNOMEx'] + ", " + self.dicionario['xNACIONALIDADEx'] + ", " + self.dicionario['xESTADO_CIVILx']\
            + ", " + self.dicionario['xPROFISSAOx'] + ", RG n° " + self.dicionario['xRGx'] + " " + self.dicionario['xEXPEDIDOR_RGx'] + ", CPF n° "\
            + self.dicionario['xCPFx'] + ", e-mail: " + self.dicionario['xEMAILx'] + ", residente e domiciliado à " + self.dicionario["xENDERECOx"]

    def _substituir(self, documento, dicionario, nomedocumento):
        #substitui os textos no documento, e salva o documento em word na pasta selecionada
        for paragrafo in documento.paragraphs:
            for chave in dicionario:
                if chave in paragrafo.text:
                    paragrafo.text = paragrafo.text.replace(chave, dicionario[chave])

        documento.save(f"Documentos Prontos\{nomedocumento} - {self.dicionario['xNOMEx']}.docx")
      
    #AQUI COMEÇAM AS FUNÇÕES DE CRIAR DOCUMENTOS - USADAS EXTERNAMENTE
    def doc(self, nomedocumento):

        #abre o arquivo    
        documento = Document(f"{self.dicionario['xESCRITORIOx']}\{nomedocumento}.docx") 

        #função para substituir os textos e salvar o documento                                 
        self._substituir(documento, self.dicionario, nomedocumento) 
     

In [4]:
#TESTE
print(PF("Terry Pratchett").dicionario["xCLIENTEx"])
Cliente = PF("Bernard Cornwell")
Cliente.dicionario
#Cliente.doc("Procuração")

Terry Pratchett, brasileiro(a), divorciado(a), cabelereiro(a), RG n° 21.897.519-3 SSP/SP, CPF n° 059.642.010-24, e-mail: n/c, residente e domiciliado à Rua 65, nº 333 - Casa 46B, Bairro Independência - 3º Complemento, CEP 74966-230, Aparecida de Goiânia - GO


{'xIDPFx': '4',
 'xNOMEx': 'Bernard Cornwell',
 'xCPFx': '291.238.740-07',
 'xRGx': '34.881.067-2',
 'xEXPEDIDOR_RGx': 'SSP/SP',
 'xESTADO_CIVILx': 'solteiro(a)',
 'xPROFISSAOx': 'empresário(a)',
 'xNACIONALIDADEx': 'brasileiro(a)',
 'xCEPx': '89239-226',
 'xLOGRADOUROx': 'Rua Irio Zanini',
 'xNUMEROx': '444',
 'xCOMPLEMENTOx': 'nan',
 'xBAIRROx': 'Pirabeiraba (Pirabeiraba)',
 'xCIDADEx': 'Joinville',
 'xUFx': 'SC',
 'xEMAILx': 'outroteste',
 'xESCRITORIOx': 'Advocacia',
 'xCADASTROx': 'cliente',
 'xCELx': 'n/c',
 'xENDERECOx': 'Rua Irio Zanini, nº 444, Bairro Pirabeiraba (Pirabeiraba), CEP 89239-226, Joinville - SC',
 'xCLIENTEx': 'Bernard Cornwell, brasileiro(a), solteiro(a), empresário(a), RG n° 34.881.067-2 SSP/SP, CPF n° 291.238.740-07, e-mail: outroteste, residente e domiciliado à Rua Irio Zanini, nº 444, Bairro Pirabeiraba (Pirabeiraba), CEP 89239-226, Joinville - SC'}

In [5]:
#Criar Classe PJ
class PJ(PF):

    def __init__(self, busca, coluna='xNOMEx', aba="PJ"):
        #carrega os atributos iniciais da classe PF
        super().__init__(busca, coluna, aba)
    
    #deixa a qualificação pronta e adiciona ao dicionário         
    def _qualificacao(self):
        self.dicionario["xCLIENTEx"] = self.dicionario['xNOMEx'] + ", pessoa jurídica inscrita no CNPJ n° " + self.dicionario['xCNPJx'] + ", com endereço à "\
            + self.dicionario['xENDERECOx'] + ", devidamente representado neste pelo sócio(a) " + self.dicionario['xADMINISTRADORx'] 
    
    #Transforma a qualificação para síndico e substitui no dicionário
    def condominio(self):
        self.dicionario["xCLIENTEx"] = self.dicionario['xNOMEx'] + ", inscrito no CNPJ n° " + self.dicionario['xCNPJx'] + ", com endereço à "\
            + self.dicionario['xENDERECOx'] + ", devidamente representado neste pelo seu síndico(a) " + self.dicionario['xADMINISTRADORx'] 


In [6]:
#TESTE
Cliente = PJ("Tyrell Corp.")
print(Cliente.dicionario["xCLIENTEx"])

Cliente.condominio()
print(Cliente.dicionario["xCLIENTEx"])

print(PJ("Nakatomi Trading Corp.").dicionario["xCLIENTEx"])

Tyrell Corp., pessoa jurídica inscrita no CNPJ n° 67.792.284/0001-77, com endereço à Travessa Manoel Reinaldo, nº 456 - Apto. 31, Bairro Jatobá, CEP 58707-425, Patos - PB, devidamente representado neste pelo sócio(a) H. P. Lovecraft
Tyrell Corp., inscrito no CNPJ n° 67.792.284/0001-77, com endereço à Travessa Manoel Reinaldo, nº 456 - Apto. 31, Bairro Jatobá, CEP 58707-425, Patos - PB, devidamente representado neste pelo seu síndico(a) H. P. Lovecraft
Nakatomi Trading Corp., pessoa jurídica inscrita no CNPJ n° 95.332.727/0001-82, com endereço à Rua da Paz, nº 349 - Apto. 41, Bairro Esplanada, CEP 64039-595, Teresina - PI, devidamente representado neste pelo sócio(a) Willian Gibson


In [11]:
class Peticoes(PF): #vai herdar as funções da classe PF

    def __init__(self, busca):
        #banco de dados
        self.bd = pd.read_excel("0. Cadastro.xlsm", aba).astype(str)
        
        #define a linha e a coluna (opcional) para busca no banco de dados
        self.linha = self._buscarnome(busca, 'xPECAx')
        
        #busca o nome da peça
        self.peca = str(self.bd.loc[self.linha, 'xPECAx'])
        
        #define a chave do dicionario
        self.chave = []

        #define o valor do dicionário (minhas chaves = xNOME_ATRIBUTOx)
        self.valores = []

        #pega a lista de atributos a serem definidos no banco de dados
        self.atributos = str(self.bd.loc[self.linha, 'xATRIBUTOSx'])
        
        #usa as outras classes para qualificar autor e réu
        self.autor = PJ("AAA").cliente
        self.reu = PF("SSSS").cliente
        self.nomereu = PF("WWWWW").nome
        
        #list comprehension = [x for x in y]
        #concatenação para acrescentar o "x", strip para tirar espaços, upper() para deixar maiuculo, split para dividir string pelo parametro ","
        self.chave = ["x" + str.strip(i).upper() + "x" for i in self.atributos.split(",")]
        self.valores = [str.strip(input(f"Defina {i} ")) for i in self.atributos.split(",")]

        #fazer dicionario com 2 listas
        self.dicionario = dict(zip(self.chave, self.valores))
        self.inserirdicionario()
       
    def inserirdicionario(self):
        self.dicionario["xAUTORx"] = self.autor
        self.dicionario["xREUx"] = self.reu
        

In [12]:
teste = Peticoes("Cobrança")
teste.atributos.split(",")
#for i in teste.atributos.split(","):
 #   print(i)
#teste.inserirvalores()
#teste.valores[1]
#teste.dicionario["xAUTORx"]
teste.dicionario



NameError: name 'aba' is not defined